In [85]:
import severability
import networkx as nx
import numpy as np
import pandas as pd

In [87]:
# Generating the graph from the edgelist, and making it undirected by summing weights in both directions
G = nx.read_edgelist("congress_network/congress.edgelist", create_using=nx.DiGraph, nodetype=int)
H = nx.create_empty_copy(G.to_undirected())
for u, v in G.edges():
    sum = G[u][v]["weight"]
    if G.has_edge(v, u):
        sum += G[v][u]["weight"]
    if (u, v) not in H.edges():
        H.add_edge(u, v, weight=sum)

In [88]:
# get adjacency matrix and convert to np.matrix
A = nx.to_numpy_array(H)
A = np.matrix(A)
assert np.array_equal(A.T, A), "Adjacency matrix is not symmetric"

# compute transition matrix
P = severability.transition_matrix(A)

In [ ]:
import json

# mapping usernames to nodes
with open("congress_network/congress_network_data.json", "r") as f:
    data = json.load(f)
    usernames = data[0]['usernameList']
    f.close()

assert len(usernames) == A.shape[0], "Number of usernames does not match the number of nodes in the graph."

In [ ]:
# Create a DataFrame to hold the usernames and severability components/values
congresspeople = pd.DataFrame(usernames, columns=["username"])
max_t = 5

for t in range(1, max_t + 1):
    congresspeople[f"Severable Component @ t = {t}"] = [set() for _ in range(len(usernames))]
    congresspeople[f"Severability @ t = {t}"] = [0.0 for _ in range(len(usernames))]

congresspeople.head()

,username,Severable Component @ t = 1,Severability @ t = 1,Severable Component @ t = 2,Severability @ t = 2,Severable Component @ t = 3,Severability @ t = 3,Severable Component @ t = 4,Severability @ t = 4,Severable Component @ t = 5,Severability @ t = 5
0,SenatorBaldwin,{},0.0,{},0.0,{},0.0,{},0.0,{},0.0
1,SenJohnBarrasso,{},0.0,{},0.0,{},0.0,{},0.0,{},0.0
2,SenatorBennet,{},0.0,{},0.0,{},0.0,{},0.0,{},0.0
3,MarshaBlackburn,{},0.0,{},0.0,{},0.0,{},0.0,{},0.0
4,SenBlumenthal,{},0.0,{},0.0,{},0.0,{},0.0,{},0.0


In [ ]:
def calculate_components(i: int, P: np.matrix, max_t = 5):
    """Calculate severable components and severability for a given congressperson."""
    print(f"Calculating components for {congresspeople['username'][i]}.")
    for t in range(1, max_t + 1):
        (c, sev) = severability.node_component(P, i, t)
        print(f"t={t}, component={[congresspeople.username[i] for i in c]}, severability={sev}")
        congresspeople.at[i, f"Severable Component @ t = {t}"] = c
        congresspeople.at[i, f"Severability @ t = {t}"] = sev

In [ ]:
# Calculate severable components and severability for each congressperson
for i in range(len(usernames)):
    calculate_components(i, P)

Calculating components for SenatorBaldwin.
t=1, component=['SenatorBaldwin', 'SenatorLankford', 'SenJohnBarrasso', 'SenBobCasey', 'SenCapito', 'SenBillCassidy', 'SenatorCardin', 'CoryBooker', 'senrobportman', 'SenSherrodBrown', 'SenDuckworth', 'SenBlumenthal', 'SenatorDurbin', 'JohnBoozman', 'SenatorCantwell', 'SenatorRisch', 'MarshaBlackburn', 'CongresswomanSC', 'SenatorSinema', 'SenJackReed', 'SenatorShaheen', 'SenAlexPadilla', 'SenJoniErnst', 'SenGillibrand', 'GerryConnolly', 'SenTedCruz', 'RepMoBrooks', 'SenatorFischer', 'SenatorBennet', 'SenatorCarper', 'ossoff', 'SenFeinstein', 'RepDonBeyer', 'SenGaryPeters', 'RepJoshG', 'SteveDaines', 'RepJerryCarl', 'SenatorBraun', 'SenRickScott', 'SenAngusKing', 'RepJacobs', 'RepCardenas', 'SenatorTimScott', 'RepLizCheney', 'RepKevinBrady', 'SenJohnKennedy', 'RepBera', 'RepNikema', 'RoyBlunt'], severability=0.46929023827146255
t=2, component=['SenatorBaldwin', 'CoryBooker', 'SenatorCardin', 'SenBobCasey', 'SenBillCassidy', 'SenJohnBarrasso', '

In [ ]:
# Save the results to a CSV file for analysis
congresspeople.to_csv("congress_severability.csv", index=False)
congresspeople.head()

,username,Severable Component @ t = 1,Severability @ t = 1,Severable Component @ t = 2,Severability @ t = 2,Severable Component @ t = 3,Severability @ t = 3,Severable Component @ t = 4,Severability @ t = 4,Severable Component @ t = 5,Severability @ t = 5
0,SenatorBaldwin,"[0, 47, 1, 14, 11, 15, 12, 6, 64, 9, 24, 4, 25...",0.469290,"[0, 6, 12, 14, 15, 1, 3, 164, 47, 13, 64, 7, 4...",0.516892,"[0, 6, 14, 12, 61, 1, 75, 64, 9, 10, 15, 24, 2...",0.490340,"[0, 6, 1, 14, 12, 61, 75, 64, 41, 10, 15, 7, 9...",0.496530,"[0, 6, 1, 14, 12, 61, 75, 207, 145, 78, 181, 1...",0.497355
1,SenJohnBarrasso,"[29, 7, 15, 165, 61, 9, 27, 125, 4, 1, 74, 28,...",0.469901,"[1, 8, 26, 5, 25, 229, 72, 7, 6, 15, 28, 14, 6...",0.523837,"[1, 25, 0, 64, 75, 7, 24, 9, 39, 12, 14, 6, 15...",0.490286,"[1, 72, 25, 0, 12, 41, 64, 7, 14, 75, 9, 15, 2...",0.496530,"[1, 72, 25, 261, 0, 14, 12, 6, 24, 7, 39, 64, ...",0.498616
2,SenatorBennet,"[2, 59, 50, 30]",0.518889,"[2, 59, 50, 30, 201, 342, 439, 5, 345, 389, 17...",0.490862,"[2, 59, 50, 30, 386, 253, 4, 3, 187, 226, 124,...",0.479445,"[2, 59, 50, 30, 386, 467, 253, 246, 4, 3, 124,...",0.486457,"[2, 59, 50, 30, 386, 307, 270, 8, 181, 372, 28...",0.491279
3,MarshaBlackburn,"[3, 66, 70]",0.521003,"[3, 66, 47, 229, 30, 48, 180, 112, 11, 45, 24,...",0.517417,"[3, 66, 70, 60, 441, 110, 105, 245, 8, 246, 36...",0.468132,"[3, 66, 70, 441, 60, 110, 105, 245, 363, 387, ...",0.481655,"[3, 66, 70, 441, 60, 110, 105, 8, 440, 246, 38...",0.487953
4,SenBlumenthal,"[4, 25, 74, 125, 72, 9]",0.452261,"[4, 24, 9, 47, 7, 23, 61, 12, 1, 75, 72, 64, 0...",0.523837,"[4, 24, 20, 9, 47, 10, 7, 1, 64, 72, 25, 12, 1...",0.510462,"[4, 24, 20, 83, 89, 9, 88, 170, 41, 64, 10, 47...",0.493798,"[4, 24, 20, 83, 90, 89, 88, 9, 191, 170, 121, ...",0.497080
